In [1]:
!pip install albumentations opencv-python matplotlib

  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached opencv-python-4.12.0.88.tar.gz (95.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyyaml-6.0.3-cp312-cp312-macosx_10_13_x86_64.whl.metadata (2.4 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv-python-headless-4.12.0.88.tar.gz (95.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 7.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2


ModuleNotFoundError: No module named 'cv2'

# Utility Functions
## Load Sample Images

In [ ]:
def load_image(path):
    """Load image in RGB format."""
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_mask(path):
    """Load mask as grayscale."""
    mask = cv2.imread(path, 0)
    return mask


## Visualization Helper

In [ ]:
def show(img, title="Image", cmap=None):
    plt.figure(figsize=(5,5))
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis("off")
    plt.show()

def show_image_mask(img, mask, title="Image + Mask"):
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title("Image")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.title("Mask")
    plt.axis("off")

    plt.show()

def draw_bbox(img, bbox):
    x_min, y_min, x_max, y_max = map(int, bbox)
    img_copy = img.copy()
    cv2.rectangle(img_copy, (x_min,y_min), (x_max,y_max), (255,0,0), 2)
    return img_copy

def draw_keypoints(img, keypoints):
    img_copy = img.copy()
    for (x,y) in keypoints:
        cv2.circle(img_copy, (int(x),int(y)), 5, (0,255,0), -1)
    return img_copy


## Pixel Level (Intensity) Transformation

In [ ]:
image = load_image("sample.jpg")   # ← replace with your file
show(image, "Original")


## Brightness Contrast

In [ ]:
transform = A.RandomBrightnessContrast(p=1.0)
augmented = transform(image=image)['image']
show(augmented, "Brightness/Contrast")

## CLAHE

In [ ]:
transform = A.CLAHE(p=1.0)
augmented = transform(image=image)['image']
show(augmented, "CLAHE")

## Gausian Noise

In [ ]:
transform = A.GaussNoise(p=1.0)
augmented = transform(image=image)['image']
show(augmented, "Gaussian Noise")

# Spatial Transformation (Image Only)
## Horizontal Flip

In [ ]:
transform = A.HorizontalFlip(p=1.0)
augmented = transform(image=image)['image']
show(augmented, "Horizontal Flip")

## Rotate

In [ ]:
transform = A.Rotate(limit=45, p=1.0)
augmented = transform(image=image)['image']
show(augmented, "Rotate")

# Spatial Transformations (Image + Mask)

In [ ]:
mask = load_mask("mask.png")   # replace with your GT mask
show_image_mask(image, mask)

## Flip + Mark

transform = A.HorizontalFlip(p=1.0)
aug = transform(image=image, mask=mask)
show_image_mask(aug['image'], aug['mask'])

## Shift Scale Rotate

transform = A.ShiftScaleRotate(
    shift_limit=0.1,
    scale_limit=0.2,
    rotate_limit=30,
    p=1.0
)

aug = transform(image=image, mask=mask)
show_image_mask(aug['image'], aug['mask'])

# Spatial Transformations (Image + Bounding Boxes)
## Dummy bounding box

bboxes = [(50, 50, 200, 200)]   # xmin, ymin, xmax, ymax
labels = [1]

img_bbox = draw_bbox(image, bboxes[0])
show(img_bbox, "Original with BBox")

## BBox Flip/ Rotate

In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=1.0)
],
bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])
)

aug = transform(image=image, bboxes=bboxes, labels=labels)
aug_img = draw_bbox(aug['image'], aug['bboxes'][0])
show(aug_img, "BBox Transformed")


# Spatial Transformations (Image + Keypoints)
## Dummy keypoints

In [ ]:
keypoints = [(120, 120), (180, 180)]

img_kp = draw_keypoints(image, keypoints)
show(img_kp, "Original with Keypoints")

## Keypoint Rotation

In [ ]:
transform = A.Compose([
    A.Rotate(limit=45, p=1.0)
],
keypoint_params=A.KeypointParams(format='xy')
)

aug = transform(image=image, keypoints=keypoints)
aug_img = draw_keypoints(aug['image'], aug['keypoints'])
show(aug_img, "Keypoint Transform")

# Segmentation Augmentation Pipeline

In [ ]:
seg_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ElasticTransform(alpha=120, sigma=12, alpha_affine=12, p=0.5),
    A.GridDistortion(p=0.3),
    A.RandomBrightnessContrast(p=0.3),
    A.Resize(256, 256),
])

: 

# Apply Segmentation Pipeline

In [ ]:
aug = seg_transform(image=image, mask=mask)
show_image_mask(aug['image'], aug['mask'], "Segmentation Augmentation Result")